In [1]:
import os
import pandas as pd
import numpy as np
# import torch
# from torch.utils.data import Dataset
# from torchvision import transforms
from sklearn.model_selection import train_test_split


In [2]:
# data_m = pd.read_csv( 'D://Dropbox//Research//Recommendation//ml-1m//movies.dat',delimiter='::', header=None, engine='python',encoding='latin-1')
# data_r = pd.read_csv( 'D://Dropbox//Research//Recommendation//ml-1m//ratings.dat',delimiter='::', header=None, engine='python',encoding='latin-1')
# data_u = pd.read_csv( 'D://Dropbox//Research//Recommendation//ml-1m//users.dat',delimiter='::', header=None, engine='python',encoding='latin-1')

data_m = pd.read_csv( '../../ml-1m/movies.dat',delimiter='::', header=None, engine='python',encoding='latin-1')
data_r = pd.read_csv( '../../ml-1m/ratings.dat',delimiter='::', header=None, engine='python',encoding='latin-1')
data_u = pd.read_csv( '../../ml-1m/users.dat',delimiter='::', header=None, engine='python',encoding='latin-1')

In [3]:
data_m.columns =['movie_id','movie_title','genre']
data_u.columns = ['user_id','gender', 'age', 'occupation', 'zip_code']
data_r.columns = ['user_id','movie_id','rating','timestamp']

In [4]:
data_m.shape, data_u.shape, data_r.shape

((3883, 3), (6040, 5), (1000209, 4))

In [5]:
# 3개의 데이터 합치기
data_combined = pd.merge(data_r, data_m, on='movie_id', how='left')
data_combined = pd.merge(data_combined, data_u, on='user_id', how='left')
data_combined

# 성별 0,1 변경
if sum(data_combined.columns == 'gender')>0:
    data_combined.rename(columns={'gender': 'sensitive'}, inplace=True)
gender_mapping = {'M': 0, 'F': 1}
data_combined['sensitive'] = data_combined['sensitive'].map(gender_mapping)
len(data_combined)

1000209

In [6]:
data_combined

,user_id,movie_id,rating,timestamp,movie_title,genre,sensitive,age,occupation,zip_code
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,1,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,1,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,0,25,6,11106
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,0,25,6,11106
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,0,25,6,11106
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,0,25,6,11106


In [7]:
# 장르 여러개 달린거 | 기준으로 나누고 원핫인코딩
unique_genres = np.unique('|'.join(data_combined['genre'].unique()).split('|'))
unique_genres

array(['Action', 'Adventure', 'Animation', "Children's", 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War',
       'Western'], dtype='<U11')

In [8]:
genre_split = data_combined['genre'].str.split('|')
genre_split

0                                       [Drama]
1              [Animation, Children's, Musical]
2                            [Musical, Romance]
3                                       [Drama]
4               [Animation, Children's, Comedy]
                           ...                 
1000204                                [Comedy]
1000205                   [Drama, Romance, War]
1000206                         [Comedy, Drama]
1000207                                 [Drama]
1000208    [Children's, Drama, Fantasy, Sci-Fi]
Name: genre, Length: 1000209, dtype: object

In [9]:
for genre in unique_genres:
    data_combined['genre_' + genre] = genre_split.apply(lambda x: int(genre in x))
    data_combined['genre_rating_' + genre] = data_combined.apply(lambda x: x.rating if genre in x['genre'].split('|') else 0, axis=1)

In [10]:
data_combined

,user_id,movie_id,rating,timestamp,movie_title,genre,sensitive,age,occupation,zip_code,...,genre_Romance,genre_rating_Romance,genre_Sci-Fi,genre_rating_Sci-Fi,genre_Thriller,genre_rating_Thriller,genre_War,genre_rating_War,genre_Western,genre_rating_Western
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,1,1,10,48067,...,0,0,0,0,0,0,0,0,0,0
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,1,1,10,48067,...,0,0,0,0,0,0,0,0,0,0
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,1,1,10,48067,...,1,3,0,0,0,0,0,0,0,0
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,1,1,10,48067,...,0,0,0,0,0,0,0,0,0,0
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,1,1,10,48067,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,0,25,6,11106,...,0,0,0,0,0,0,0,0,0,0
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,0,25,6,11106,...,1,5,0,0,0,0,1,5,0,0
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,0,25,6,11106,...,0,0,0,0,0,0,0,0,0,0
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,0,25,6,11106,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# 한 사람이 각 장르를 몇번 봤는지 count하고 rating을 평균내서 genre_rating_으로 column 만듦
genre_dict = dict(('genre_' + genre, 'sum') for genre in unique_genres)
genre_rating_dict = dict(('genre_rating_' + genre, lambda x: x[x!=0].mean()) for genre in unique_genres)

In [12]:
def iden(x):
    return list(set(x))[0]

agg_dict = {
    'movie_id': 'count', 
    'movie_title': ' '.join, 
    'sensitive': iden, 
    'age': iden, 
    'occupation': iden, 
    'zip_code': iden
}

d = dict()
d.update(agg_dict)
d.update(genre_dict)
d.update(genre_rating_dict)

df = data_combined.groupby(by='user_id').agg(d)

In [13]:
df

,movie_id,movie_title,sensitive,age,occupation,zip_code,genre_Action,genre_Adventure,genre_Animation,genre_Children's,...,genre_rating_Fantasy,genre_rating_Film-Noir,genre_rating_Horror,genre_rating_Musical,genre_rating_Mystery,genre_rating_Romance,genre_rating_Sci-Fi,genre_rating_Thriller,genre_rating_War,genre_rating_Western
user_id,,,,,,,,,,,,,,,,,,,,,
1,53,One Flew Over the Cuckoo's Nest (1975) James a...,1,1,10,48067,5,5,18,20,...,4.00,NaN,NaN,4.285714,NaN,3.666667,4.333333,3.666667,5.000000,NaN
2,129,"Shine (1996) Verdict, The (1982) Shall We Danc...",0,56,16,70072,56,19,0,0,...,3.00,4.000000,3.000000,NaN,3.333333,3.708333,3.588235,3.483871,3.733333,4.333333
3,51,"Animal House (1978) Full Monty, The (1997) Mis...",0,25,15,55117,23,25,3,3,...,4.50,NaN,2.666667,4.000000,3.000000,3.800000,3.833333,3.800000,4.000000,4.666667
4,21,"Hustler, The (1961) Star Wars: Episode VI - Re...",0,45,7,02460,19,6,0,1,...,4.50,NaN,4.333333,NaN,NaN,4.000000,3.555556,3.500000,3.333333,4.500000
5,198,Who Framed Roger Rabbit? (1988) Gods and Monst...,0,25,20,55455,31,9,4,6,...,NaN,4.000000,2.800000,3.333333,3.125000,3.100000,3.066667,2.846154,3.500000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,888,"Wedding Gift, The (1994) Spanking the Monkey (...",1,25,15,32603,154,83,34,54,...,3.00,4.058824,2.986486,3.709677,3.411765,3.352459,2.834320,3.142857,3.785714,3.642857
6037,202,"Terminator 2: Judgment Day (1991) Insider, The...",1,45,1,76006,28,9,1,6,...,4.25,3.444444,4.111111,4.000000,3.692308,3.681818,3.692308,3.705882,4.000000,3.750000
6038,20,Walkabout (1971) Gone with the Wind (1939) Har...,1,56,1,14706,2,1,3,1,...,NaN,NaN,2.500000,NaN,NaN,4.166667,4.000000,NaN,4.000000,NaN


In [14]:
# df.to_csv('D://data_combined_new.csv')

In [15]:
# df.to_csv('data_combined_new.csv')